In [5]:
import os
import os
import json
from llama_index.core import SimpleDirectoryReader
import ollama
from groq import Groq
from langchain import PromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from pathlib import Path
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pickle
import time

/run/media/aasiku/Vault/GlobalPythonEnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Importiong api keys
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API")

In [7]:
Parent_directory = Path('Papers')
File_paths = Path.rglob(Parent_directory, '*.pdf')

In [8]:
File_paths

<generator object Path.rglob at 0x71a1ff2317a0>

In [6]:
File_paths = [str(i) for i in File_paths]

In [7]:
File_paths

['Papers/Attention_is_all_you_need.pdf',
 'Papers/DCGAN-notes.pdf',
 'Papers/DeepSORT.pdf',
 'Papers/Fast-RCNN.pdf',
 'Papers/GANs_Paper.pdf',
 'Papers/Image_Augmentation_IllusionCraft.pdf',
 'Papers/Mamba.pdf',
 'Papers/Mask RCNN.pdf',
 'Papers/Mismatching_images___Keeping_a_check_on_the_generator (1).pdf',
 'Papers/SORT.pdf',
 'Papers/StackGAN.pdf',
 'Papers/StackGAN_original_paper.pdf',
 'Papers/The Power of Linear Recurrent Neural Networks.pdf',
 'Papers/Variational Auto encoders.pdf',
 'Papers/Word2Vec Paper.pdf',
 'Papers/ESC201/Lecture10.pdf',
 'Papers/ESC201/Lecture11.pdf',
 'Papers/ESC201/Lecture12a.pdf',
 'Papers/ESC201/Lecture12b.pdf',
 'Papers/ESC201/Lecture13.pdf',
 'Papers/ESC201/Lecture15.pdf',
 'Papers/ESC201/Lecture16.pdf',
 'Papers/ESC201/Lecture17.pdf',
 'Papers/ESC201/Lecture18.pdf',
 'Papers/ESC201/Lecture19.pdf',
 'Papers/ESC201/Lecture20_21.pdf',
 'Papers/ESC201/Lecture22.pdf',
 'Papers/ESC201/Lecture5.pdf',
 'Papers/ESC201/Lecture6_7.pdf',
 'Papers/ESC201/Lectur

#### Using LangSimth for monitoring

In [11]:
# Using langsmith to monitor the progress of the summarization
#! If you want to use langsmith, please set the environment variable LANGCHAIN_API to your langsmith api key
if os.environ.get("LANGCHAIN_API")!="":
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"]=os.environ.get("LANGCHAIN_API")
    os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"]="LearnLang"

### Summarization

In [9]:
# Functionizing till split_chunks
def input_llm_pdf(File_path): #!Pass the file path of the pdf, this function will return the split_chunks
    # Reading the pdf file
    pdfreader = PdfReader(File_path)
    text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            text += content
    # Converting the text of the pdf of Document object
    docs = [Document(page_content=text)]
    docs
    ## Splittting the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300000, chunk_overlap=200) # 300k characters per chunk, or nearly 75,000 tokens
    chunks = text_splitter.create_documents([text])
    split_chunks = []
    for x in range(0,len(chunks),12):
        split_chunks.append(chunks[x:x+12])
    return split_chunks

In [10]:
# Functionizing till split_chunks
def input_llm_pdf_hf(File_path): #!Pass the file path of the pdf, this function will return the split_chunks
    # Reading the pdf file
    pdfreader = PdfReader(File_path)
    text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            text += content
    # Converting the text of the pdf of Document object
    docs = [Document(page_content=text)]
    docs
    ## Splittting the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=70000, chunk_overlap=200) # 300k characters per chunk, or nearly 75,000 tokens
    chunks = text_splitter.create_documents([text])
    return chunks

#### Prompts for summarizing

In [9]:
chunks_prompt="""
Sumarise the bellow document, it will be used to later match the document with a user query, so capture the most important informations.
Document:`{text}'.Also in a line return the keywords in the document, these keywords must include the vast majority of the important points in the document.Use the maximum number of characters to express the summary under 20,000 characters.
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [10]:
final_combine_prompt='''
Provide a final summary of the entire document with these important points, this will be used to match the document with a user query, so capture the most important informations.Also in a line return the keywords in the document, these keywords must include the vast majority of the important points in the document.Use the maximum number of characters to express the summary under 20,000 characters.
Document: `{text}`
'''
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.25,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size = 15,  # Controls the maximum burst size.
)
GEMINI_API_KEY=os.environ.get("GEMINI_API")
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=GEMINI_API_KEY,rate_limiter=rate_limiter)

/tmp/ipykernel_6090/925386506.py:4: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [14]:
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
import os

In [15]:
# if os.getenv("HUGGINGFACE_API"):
#     os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACE_API")

In [21]:
# llm = HuggingFaceEndpoint(
#     repo_id="HuggingFaceH4/zephyr-7b-beta",
#     # model_id=model_id,
#     task="text-generation",
#     max_new_tokens=2048,
#     do_sample=False,
#     repetition_penalty=1.03,
# )

# # pipeline = transformers.pipeline(
# #     "text-generation",
# #     model=model_id,
# #     model_kwargs={"torch_dtype": torch.bfloat16},
# #     device_map="auto",
# # )
# chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/aasiku/.cache/huggingface/token
Login successful


In [18]:
import concurrent.futures
import time
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"]=os.environ.get("LANGCHAIN_API")
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="LearnLang"
## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200000, chunk_overlap=200) # 300k characters per chunk, or nearly 75,000 tokens
# summary_chain = []
summary_chain = load_summarize_chain(
    llm=gemini_llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)



Processing file: Papers/Attention_is_all_you_need.pdf
Index 1 of 80 completed


Processing file: Papers/DCGAN-notes.pdf
Index 2 of 80 completed


Processing file: Papers/DeepSORT.pdf
Index 3 of 80 completed


Processing file: Papers/Fast-RCNN.pdf
Index 4 of 80 completed


Processing file: Papers/GANs_Paper.pdf
Index 5 of 80 completed


Processing file: Papers/Image_Augmentation_IllusionCraft.pdf
Index 6 of 80 completed


Processing file: Papers/Mamba.pdf
Index 7 of 80 completed


Processing file: Papers/Mask RCNN.pdf
Index 8 of 80 completed


Processing file: Papers/Mismatching_images___Keeping_a_check_on_the_generator (1).pdf
Index 9 of 80 completed


Processing file: Papers/SORT.pdf
Index 10 of 80 completed


Processing file: Papers/StackGAN.pdf
Index 11 of 80 completed


Processing file: Papers/StackGAN_original_paper.pdf
Index 12 of 80 completed


Processing file: Papers/The Power of Linear Recurrent Neural Networks.pdf
Index 13 of 80 completed


Processing file: Papers/Variationa

/tmp/ipykernel_6090/1762470873.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output += summary_chain.run(chunks)
/tmp/ipykernel_6090/1762470873.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output += summary_chain.run(chunks)
/tmp/ipykernel_6090/1762470873.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output += summary_chain.run(chunks)
/tmp/ipykernel_6090/1762470873.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output += summary_chain.run(chunks)
/tmp/ipykernel_6090/1762470873.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 


Processed file: Papers/Fast-RCNN.pdf

Processing file: Papers/ESC201/Lecture10.pdf
Index 16 of 80 completed


Processed file: Papers/DeepSORT.pdf

Processing file: Papers/ESC201/Lecture11.pdf
Index 17 of 80 completed


Processed file: Papers/Word2Vec Paper.pdf

Processing file: Papers/ESC201/Lecture12a.pdf
Index 18 of 80 completed


Processed file: Papers/GANs_Paper.pdf

Processing file: Papers/ESC201/Lecture12b.pdf
Index 19 of 80 completed


Processed file: Papers/Attention_is_all_you_need.pdf

Processing file: Papers/ESC201/Lecture13.pdf
Index 20 of 80 completed


Processed file: Papers/SORT.pdf

Processing file: Papers/ESC201/Lecture15.pdf
Index 21 of 80 completed


Processed file: Papers/ESC201/Lecture10.pdf

Processing file: Papers/ESC201/Lecture16.pdf
Index 22 of 80 completed


Processed file: Papers/Mask RCNN.pdf

Processing file: Papers/ESC201/Lecture17.pdf
Index 23 of 80 completed


Processed file: Papers/StackGAN_original_paper.pdf

Processing file: Papers/ESC201/Lecture18.p

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



Processed file: Papers/ESC201/Lecture9.pdf

Processing file: Papers/ESO201/Lecture 15.pdf
Index 41 of 80 completed


Processed file: Papers/ESC201/Lecture19.pdf

Processing file: Papers/ESO201/Lecture 16.pdf
Index 42 of 80 completed


Processed file: Papers/ESC201/Lecture6_7.pdf

Processing file: Papers/ESO201/Lecture 17.pdf
Index 43 of 80 completed


Processed file: Papers/ESO201/Lecture 1.pdf

Processing file: Papers/ESO201/Lecture 18.pdf
Index 44 of 80 completed


Processed file: Papers/ESO201/Lecture 13.pdf

Processing file: Papers/ESO201/Lecture 19.pdf
Index 45 of 80 completed


Processed file: Papers/ESO201/Lecture 11.pdf

Processing file: Papers/ESO201/Lecture 19_0.pdf
Index 46 of 80 completed


Processed file: Papers/ESC201/Lecture20_21.pdf

Processing file: Papers/ESO201/Lecture 2.pdf
Index 47 of 80 completed


Processed file: Papers/ESC201/Lecture16.pdf

Processing file: Papers/ESO201/Lecture 3.pdf
Index 48 of 80 completed


Processed file: Papers/ESO201/Chem Thermo 2.pdf

P

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



Processed file: Papers/MSO notes/MSO202Lect1.pdf

Processing file: Papers/MSO notes/MSO202Lect4.pdf
Index 69 of 80 completed


Processed file: Papers/ESO201/Lecture 6.pdf

Processing file: Papers/MSO notes/MSO202Lect5.pdf
Index 70 of 80 completed


Processed file: Papers/ESO201/Lecture 2.pdf

Processing file: Papers/MSO notes/MSO202Lect6.pdf
Index 71 of 80 completed


Processed file: Papers/ESO201/Lecture 15.pdf

Processing file: Papers/MSO notes/MSO202Lect7.pdf
Index 72 of 80 completed


Processed file: Papers/ESO201/Lecture 19.pdf

Processing file: Papers/MSO notes/MSO202Lect8.pdf
Index 73 of 80 completed


Processed file: Papers/MSO notes/MSO202Lect14.pdf

Processing file: Papers/MSO notes/MSO202Lect9.pdf
Index 74 of 80 completed


Processed file: Papers/MSO notes/MSO202Lect2.pdf

Processing file: Papers/MSO notes/MSO202M_Endsem_Y22.pdf
Index 75 of 80 completed


Processed file: Papers/MSO notes/MSO202Lect6.pdf

Processing file: Papers/ESC201/PYQ/1820448fe9c9eaac0b9d32f1384599b7c8f

FloatObject (b'0..992') invalid; use 0.0 instead
FloatObject (b'1.50.975') invalid; use 0.0 instead
FloatObject (b'0.1.50.911') invalid; use 0.0 instead
FloatObject (b'1.00.962801') invalid; use 0.0 instead
FloatObject (b'0.999-0.02930.999-05') invalid; use 0.0 instead
FloatObject (b'0..92') invalid; use 0.0 instead
FloatObject (b'0.56-0.0611') invalid; use 0.0 instead
FloatObject (b'-0-0.99220.59') invalid; use 0.0 instead
FloatObject (b'0.46681.0167') invalid; use 0.0 instead
FloatObject (b'0..31.011') invalid; use 0.0 instead
FloatObject (b'1.28.79.46681.28.79.4') invalid; use 0.0 instead
FloatObject (b'0.55.79.-46681.0..31.-011') invalid; use 0.0 instead
FloatObject (b'1.-011') invalid; use 0.0 instead
FloatObject (b'1.0.-46681..029') invalid; use 0.0 instead
FloatObject (b'0.4360-1.045') invalid; use 0.0 instead
FloatObject (b'2.003820.59') invalid; use 0.0 instead
FloatObject (b'0..222.0038205559') invalid; use 0.0 instead
FloatObject (b'0.4360-15559') invalid; use 0.0 instead
Fl

File at index 56 raised an exception: Invalid Elementary Object starting with b'*' @927187: b'..550603*\nQ2.468 c\n3*\nQ2.400.3*\nQ2.-468 c\n55337.-.95060.95060-.950600.f*6750-.95'

Processed file: Papers/MSO notes/MSO202Lect15.pdf

Processed file: Papers/MSO notes/MSO202Lect8.pdf

Processed file: Papers/MSO notes/MSO202Lect3.pdf

Processed file: Papers/ESC201/PYQ/1820448fe9c9eaac0b9d32f1384599b7c8fab6b5697e7748eafba52c66470b96_Midsem sol_to_showw.pdf

Processed file: Papers/ESC201/PYQ/Lec14_MOSFET_I-V_characteristics.pdf

Processed file: Papers/ESC201/PYQ/Midsem sol_to_showw.pdf

Processed file: Papers/MSO notes/MSO202Lect13.pdf

Processed file: Papers/MSO notes/MSO202Lect9.pdf

Processed file: Papers/ESC201/PYQ/Midsem 2020 II.pdf

Processed file: Papers/ESC201/PYQ/Midsem 2020 I.pdf

Processed file: Papers/ESO201/thermo_solution_8thedi.pdf


In [12]:
#  llama-3.1-70b-versatile
def process_file(index, file_path, requests_in_this_minute, start_time):
    output = ''
    file_path = str(File_paths[index])
    print(f"\nProcessing file: {file_path}")
    print(f"Index {index+1} of {len(File_paths)} completed\n")  
    split_chunks = input_llm_pdf(file_path)
    for chunks in split_chunks:
        output += summary_chain.run(chunks)
        endtime = time.time()
    # output_doc = Document(page_content=output)
    chunks = text_splitter.create_documents([output])
    requests_in_this_minute += len(chunks)

    final_summary = summary_chain.run(chunks)
    final_summary = f"File_path: {File_paths[index]} \n"+final_summary
    index+=1
    # summarized_documents.append([Document(page_content=final_summary)])
    print(f"\nProcessed file: {file_path}")
    return final_summary
def process_file_hf(index, file_path, requests_in_this_minute, start_time):
    output = ''
    file_path = str(File_paths[index])
    print(f"\nProcessing file: {file_path}")
    print(f"Index {index+1} of {len(File_paths)} completed\n")  
    chunks = input_llm_pdf_hf(file_path)
    output += summary_chain.run(chunks)
    endtime = time.time()
    # output_doc = Document(page_content=output)
    chunks = text_splitter.create_documents([output])
    requests_in_this_minute += len(chunks)

    final_summary = summary_chain.run(chunks)
    final_summary = f"File_path: {File_paths[index]} \n"+final_summary
    index+=1
    # summarized_documents.append([Document(page_content=final_summary)])
    print(f"\nProcessed file: {file_path}")
    return final_summary



def parallel_process_files():
    # Initialize some shared variables
    requests_in_this_minute = 0
    start_time = time.time()
    summarized_documents = []
    
    # Create a ThreadPoolExecutor for parallelizing the operations
    with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
        future_to_index = {executor.submit(process_file, i, File_paths[i], requests_in_this_minute, start_time): i for i in range(len(File_paths))}
        
        # Collect results
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                result = future.result()
                if result is not None:
                    summarized_documents.append([Document(page_content=result)])
            except Exception as e:
                print(f"File at index {index} raised an exception: {e}")
    
    return summarized_documents



In [ ]:

# Start the parallel processing
summarized_documents = parallel_process_files()


In [19]:
summarized_documents

[[Document(metadata={}, page_content="File_path: Papers/Fast-RCNN.pdf \n## Summary:\n\nFast R-CNN is a groundbreaking object detection method that significantly outperforms previous approaches like R-CNN and SPPnet by achieving faster training and testing speeds while simultaneously boosting detection accuracy. It introduces single-stage training, where object classification and bounding-box refinement are learned jointly using a multi-task loss function, optimizing both tasks. Back-propagation through RoI pooling enables efficient learning by updating all network layers, eliminating the need for feature caching.\n\nFast R-CNN demonstrates impressive speed improvements: 213 times faster testing and 9 times faster training compared to R-CNN. It also achieves a higher mean Average Precision (mAP) on the PASCAL VOC 2012 dataset, reaching 66% compared to R-CNN's 62%.\n\nThe paper explores various design choices, revealing that multi-task training enhances both classification and localizati

### GROQ

In [13]:
from langchain_groq import ChatGroq
model_id =  "llama-3.1-70b-versatile"
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.067,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size = 15,  # Controls the maximum burst size.
)
llm = ChatGroq(
    model=model_id,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY,
    # other params...
)

NameError: name 'InMemoryRateLimiter' is not defined

In [41]:
GROQ_API_KEY=os.environ.get("GROQ_WHISPER_KEY")
GROQ_API_KEY

'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'

In [44]:
llm.invoke("Hello, world!")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_time': 0.04, 'prompt_time': 0.009308826, 'queue_time': 0.004998464000000001, 'total_time': 0.049308826}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-6c613c6a-39f4-4bf1-8bb8-184bc2dde7c9-0', usage_metadata={'input_tokens': 39, 'output_tokens': 10, 'total_tokens': 49})

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=GEMINI_API_KEY)

In [15]:
len(summarized_documents),len(File_paths)

NameError: name 'summarized_documents' is not defined

In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
GEMINI_API_KEY = os.environ.get("GEMINI_API")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=GEMINI_API_KEY)

# Create a Chroma instance
vector_store = Chroma(embedding_function=embeddings, persist_directory="Summaries-2")

# Load documents (replace with your own document loading logic)


/tmp/ipykernel_11015/2633589271.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings, persist_directory="Summaries-2")


In [17]:

#! Saving the summarized documents, for future use
# with open('summaries.txt', 'w') as f:
#     for item in summarized_documents:
#         f.write("%s\n" % item)

In [18]:
for x in summarized_documents: 
    vector_store.add_documents(x)


NameError: name 'summarized_documents' is not defined

In [19]:
retriever_or = vector_store.as_retriever()

In [20]:
retriever_or.invoke("What is exergy?")

[Document(metadata={}, page_content='File_path: Papers/ESO201/Chem Thermo 1.pdf \n## Summary:\n\nThis document provides a comprehensive overview of fundamental concepts in chemical thermodynamics, focusing on predicting the spontaneity of chemical reactions and processes. It begins by introducing internal energy, its relationship to heat transfer, work, and matter exchange, and the first law of thermodynamics, which states that the energy of an isolated system remains constant. \n\nThe document explores pressure-volume work, relevant for gases, and the concept of reversible processes involving infinitesimal changes in pressure and volume. It then delves into heat capacity, the amount of heat required to raise the temperature of a system, distinguishing between specific and molar heat capacities. \n\nThe document introduces entropy, a thermodynamic state function representing the randomness or disorder of a system, highlighting factors that affect it, such as randomness, heat absorption

In [21]:
chunks_prompt="""
Using the Context bellow answer the question {question}, mention the path of the MOST RELAVENT documents.

NOTE: The contexts are summaries of maybe very large documents, so scrutinize it well and at the end of each summary keywords are also mentioned, use these also to answer the question.RETURN ONLY PATHS TO THE MOST RELEVANT DOCUMENTS
Context:
{text}

IMPORTANT: The answer should be in the following format:
RETURN ONLY JSON DATA NOTHING ELSE
```
    {{
    "files": [
        {{
        "file_path": "path to the file ",
        "reason":"How is the file relevant to the question"
        }}
    ]
    }}
    ```
"""
RAG_prompt_template=PromptTemplate(input_variables=['text','question'],
                                    template=chunks_prompt)

In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
# question = "Explain me what is the attention mechanism in 100 words"
# context = retriever.invoke(question)
rag_chain = (
    {"text": retriever_or | format_docs, "question": RunnablePassthrough()}
    | RAG_prompt_template
    | gemini_llm
    | StrOutputParser()
)


NameError: name 'gemini_llm' is not defined

In [190]:
question = "What is entropy?"


In [1]:
response = ""
for s in rag_chain.stream(question):
    print(s)
    response += s

NameError: name 'rag_chain' is not defined

In [192]:
response

'```json\n{\n"files": [\n{\n"file_path": "Papers/ESO201/Chem Thermo 1.pdf",\n"reason": "This document provides a comprehensive overview of chemical thermodynamics, including entropy, its definition, and its role in predicting the spontaneity of reactions. It also explores the third law of thermodynamics, which is related to entropy at absolute zero."\n},\n{\n"file_path": "Papers/ESO201/Lecture 15.pdf",\n"reason": "This document focuses on the concept of entropy, defining it as a thermodynamic property representing a system\'s energy change due to heat transfer. It explores entropy\'s role in reversible and irreversible processes, aligning with the second law of thermodynamics."\n},\n{\n"file_path": "Papers/ESO201/Lecture 16.pdf",\n"reason": "This document delves into entropy change in thermodynamic systems, focusing on pure substances and ideal gases. It explores the Clausius equation, the Increase of entropy principle, and the Gibbs equation, all of which are related to entropy calcul

In [152]:
formated_response = response.strip('`').split('json')[1]

In [153]:
json_response = json.loads(formated_response)

In [154]:
json_response

{'files': [{'file_path': 'Papers/ESO201/thermo_solution_8thedi.pdf',
   'reason': 'This document provides a comprehensive overview of thermodynamics concepts, including exergy analysis, which directly addresses the question of what exergy is.'}]}

In [155]:
File_paths_relavent = [x['file_path'] for x in json_response['files']]
File_paths_relavent

['Papers/ESO201/thermo_solution_8thedi.pdf']

In [156]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=GEMINI_API_KEY)

In [157]:
# Functionizing till split_chunks for vectorization
def input_llm_embeddings_pdf(File_path): #!Pass the file path of the pdf, this function will return the split_chunks
    # Reading the pdf file
    pdfreader = PdfReader(File_path)
    text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            text += content
    # Converting the text of the pdf of Document object
    docs = [Document(page_content=text)]
    docs
    ## Splittting the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200) # 300k characters per chunk, or nearly 75,000 tokens
    chunks = text_splitter.create_documents([text])
    print(chunks)
    split_chunks = []
    for x in range(0,len(chunks),1400):
        split_chunks.append(chunks[x:x+1400])
    return split_chunks

In [159]:
!mkdir -p VectorDatabases

I0000 00:00:1728561119.819930    6090 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [160]:
File_paths_relavent

['Papers/ESO201/thermo_solution_8thedi.pdf']

In [169]:
index = 0
path_vector_database = Path('VectorDatabases')
while index < len(File_paths_relavent):
    file_path = File_paths_relavent[index]
    print(f"Processing file: {file_path}")
    print(f"Index {index+1} of {len(File_paths_relavent)} completed")

    path_vector_db_folder = Path.joinpath(path_vector_database,file_path)
    # path_vector_db_folder.mkdir(parents)
    path_vector_db_folder.mkdir(parents=True, exist_ok=True)
    if not path_vector_db_folder.exists():
        split_chunks = input_llm_embeddings_pdf(file_path)
        try:
            for chunks in split_chunks:
                vector_store = Chroma(embedding_function=embeddings, persist_directory=str(path_vector_db_folder))
                # print(f"Adding document {docs}")
                vector_store.add_documents(chunks)
        except Exception as e:
            print(f"Exception: {e}")
            print("Resource exhausted, waiting for 60 seconds...")
            index -= 1
            time.sleep(60)
    index += 1


Processing file: Papers/ESO201/thermo_solution_8thedi.pdf
Index 1 of 1 completed


In [170]:
retriever = vector_store.as_retriever()

In [173]:
chunks_prompt="""
Using the Context bellow answer the question {question}, also mention the page number, name of the doc, etc relavent details at /the end of the response.Try to gatter as much information as possible from the context, and answer on the basis of that, if the context is not enough to answer the question, then mention that in the response, along with what you were able to gather from the context.

Context:
{text}
"""
RAG_prompt_template=PromptTemplate(input_variables=['text','question'],
                                    template=chunks_prompt)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
context = retriever.invoke(question)
rag_chain = (
    {"text": retriever | format_docs, "question": RunnablePassthrough()}
    | RAG_prompt_template
    | gemini_llm
    | StrOutputParser()
)

from langchain_google_genai import ChatGoogleGenerativeAI
GEMINI_API_KEY=os.environ.get("GEMINI_API")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=GEMINI_API_KEY)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]=os.environ.get("LANGCHAIN_API")
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="LearnLang"

In [174]:
# context = list()
# index = 0
# path_vector_database = Path('VectorDatabases')
# while index < len(File_paths_relavent):
#     file_path = File_paths_relavent[index]
#     print(f"Processing file: {file_path}")
#     print(f"Index {index+1} of {len(File_paths_relavent)} completed")

#     path_vector_db_folder = Path.joinpath(path_vector_database,file_path)
#     path_vector_db = Path.joinpath(path_vector_db_folder, "faiss_index.index")
#     path_docstore = Path.joinpath(path_vector_db_folder, "docstore.pkl")
#     loaded_faiss_index = faiss.read_index(str(path_vector_db))
#     # Load the docstore and index_to_docstore_id
#     with open(str(path_docstore), 'rb') as f:
#         loaded_docstore = pickle.load(f)

#     with open(str(path_index_to_docstore_id), 'rb') as f:
#         loaded_index_to_docstore_id = pickle.load(f)


#     vector_db_loaded = FAISS(
#         embeddings.embed_query, 
#         loaded_faiss_index, 
#         loaded_docstore, 
#         loaded_index_to_docstore_id
#     )
#     retriever = vector_db_loaded.as_retriever()
#     context.extend(retriever.invoke(question))
#     index += 1

In [175]:
for s in rag_chain.stream(question):
    print(s)

The
 context provided does not directly define exergy. However, it does mention it in
 relation to the work potential of various systems, such as blowing air for windmills and
 water stored at a high elevation for a hydroelectric power plant. 

Here's what we can gather from the context:

* **Exergy is related to the work
 potential of a system.** This means it represents the maximum amount of useful work that can be extracted from a system as it comes to equilibrium with its surroundings.
* **
Exergy is a function of the state of the surroundings as well as the state of the system.** This implies that exergy is not just an intrinsic property of the system itself, but also depends on the environment it is in.
*
 **Exergy can be expressed in units of energy.** The context shows examples using kJ/kg and kWh.

**Based on the context, we can infer that exergy is a measure of the potential for useful work that a system possesses
, taking into account both the system's state and the state of

In [177]:
import streamlit as st

# Assuming rag_chain is already defined and set up
# Example:
# rag_chain = YourChainObject()

def stream_question(question):
    # Stream each result from the chain
    for s in rag_chain.stream(question):
        # Print each result in the Streamlit app
        st.write(s)  # You can use st.text(s) or st.markdown(s) for different formatting

# Streamlit UI
st.title("Streaming Question Answering")

# Input question from user
question = st.text_input("Enter your question:")

# When a question is provided, stream the output


2024-10-10 17:34:47.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.735 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 17:34:47.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [49]:
# file_path = File_paths_relavent[0]
# def return_context(question):
#     index = 0
#     while index < len(File_paths_relavent):
#         file_path = File_paths_relavent[index]
#         print(f"Processing file: {file_path}")
#         print(f"Index {index+1} of {len(File_paths_relavent)} completed")

#         path_vector_db_folder = Path.joinpath(path_vector_database,file_path)
#         path_vector_db = Path.joinpath(path_vector_db_folder, "faiss_index.index")
#         path_docstore = Path.joinpath(path_vector_db_folder, "docstore.pkl")
#         loaded_faiss_index = faiss.read_index(str(path_vector_db))
#         # Load the docstore and index_to_docstore_id
#         with open(str(path_docstore), 'rb') as f:
#             loaded_docstore = pickle.load(f)

#         with open(str(path_index_to_docstore_id), 'rb') as f:
#             loaded_index_to_docstore_id = pickle.load(f)


#         vector_db_loaded = FAISS(
#             embeddings.embed_query, 
#             loaded_faiss_index, 
#             loaded_docstore, 
#             loaded_index_to_docstore_id
#         )
#         retriever = vector_db_loaded.as_retriever()
#         context.extend(retriever.invoke(question))
#         index +=1 
#     return context

In [72]:
chunks_prompt="""
Using the Context bellow answer the question {question}, also mention the page number, name of the doc, etc relavent details at /the end of the response.

Context:
{text}
"""
RAG_prompt_template=PromptTemplate(input_variables=['text','question'],
                                    template=chunks_prompt)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
context = retriever.invoke(question)
rag_chain = (
    {"text": return_context , "question": RunnablePassthrough()}
    | RAG_prompt_template
    | llm
    | StrOutputParser()
)

NameError: name 'return_context' is not defined

In [52]:
response = rag_chain.invoke(question)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Processing file: Papers/Attention_is_all_you_need.pdf
Index 1 of 1 completed


In [53]:
response

'The attention mechanism in a neural network allows the model to focus on specific parts of the input sequence when processing information. It assigns weights to different parts of the input, giving more importance to relevant elements. This is done by calculating a "compatibility function" between a "query" and a set of "key-value" pairs.  The weights are then used to compute a weighted sum of the "values," yielding an output that reflects the attended information.\n\nThis explanation is from the document titled "Attention is All You Need", on page 3. \n'